
### Step 1: Connect to ADLS

Since we are using Azure Credentials Passthrough connection method, we just need to use the custom access token and mount the folder that has all our TestStand files. 

In [ ]:
# Declare script variables
database_name = "dbo"

bronze_source = "REDACTED"
bronze_target = bronze_source + database_name
bronze_mnt = "/mnt/bronze/" + database_name

silver_source = "REDACTED"
silver_target = silver_source + database_name
silver_mnt = "/mnt/silver/" + database_name

gold_source = "REDACTED"
gold_target = gold_source + database_name
gold_mnt = "/mnt/gold/" + database_name

# Azure Credentials Passthrough configuration.
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}



### Step 2: Mount bronze container


In [ ]:
# Add bronze_mnt if not already mounted.
if any(mount.mountPoint == bronze_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=bronze_source, mount_point=bronze_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(bronze_target)


### Step 3: Mount silver container


In [ ]:
# Add silver_mnt if not already mounted.
if any(mount.mountPoint == silver_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
        source=silver_source, mount_point=silver_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(silver_target)


### Step 4: Mount gold container


In [ ]:
# Add gold_mnt if not already mounted.
if any(mount.mountPoint == gold_mnt for mount in dbutils.fs.mounts()):
    pass
else:
    dbutils.fs.mount(
      source=gold_source, mount_point=gold_mnt, extra_configs=configs
    )

# List files in directory
dbutils.fs.ls(gold_target)

[]


### Step 5: Setup Database and Extract Data to Tables in Databricks

We instantiate a DBFS database and USE it. Then we iterate over the TSDB directory at that level and create a table for each CSV file in that mount point. 


In [ ]:
# Create and use DATABASE [drop IF ALREADY EXISTS]
spark.sql(f"drop database IF EXISTS {database_name} cascade ")
spark.sql(f"create database {database_name}")
spark.sql(f"use {database_name}")

# Create all tables into DBFS database
tables_list = dbutils.fs.ls(bronze_target)

for table in tables_list:
    table_name = table.name.strip(".parquet")
    spark.sql(f"DROP TABLE IF EXISTS `{table_name}`")
    spark.sql(f"""
              CREATE TABLE `{table_name}` 
              USING parquet 
              OPTIONS(path '{table.path}', header 'true', inferschema 'true')
              """)
